In [35]:
#Libraries
import numpy as np
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import MinMaxScaler, StandardScaler

from sklearn.neighbors import KNeighborsRegressor
from sklearn.neighbors import KNeighborsClassifier

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix


import pandas as pd
import glob
import os
import re



In [1]:
import pandas as pd
import glob
import os
import re
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Specify the folder path where CSV files are stored
folder_path = 'data/'

# Use glob to find all CSV files in the specified folder
all_files = glob.glob(os.path.join(folder_path, "*.csv"))

# Use a list comprehension to read each CSV file into a DataFrame and ensure 'Date' is string
df_list = []
for file in all_files:
    try:
        # Read each CSV and convert 'Date' to string format
        buli_df = pd.read_csv(file, encoding='ISO-8859-1', dtype={'Date': str})
        df_list.append(buli_df)
    except pd.errors.ParserError as e:
        print(f"ParserError parsing {file}: {e}")
    except UnicodeDecodeError as e:
        print(f"UnicodeDecodeError in {file}: {e}")

# Concatenate all DataFrames in the list into a single DataFrame
buli_df = pd.concat(df_list, ignore_index=True)

# Standardize and parse the 'Date' column
buli_df['Date'] = buli_df['Date'].str.strip()  # Remove extra whitespace
buli_df['Date'] = buli_df['Date'].replace(r'[/-]', '-', regex=True)  # Replace separators with '-'

# Attempt to parse dates as `dayfirst` and handle both `dd/mm/yyyy` and `dd/mm/yy`
buli_df['Date'] = buli_df['Date'].apply(lambda x: re.sub(r'(\d{2}/\d{2}/)(\d{2})$', r'\120\2', x))
buli_df['Date'] = pd.to_datetime(buli_df['Date'], dayfirst=True, errors='coerce')

# Check for any remaining NaT values in 'Date' after parsing
missing_dates = buli_df[buli_df['Date'].isna()]
if not missing_dates.empty:
    print("Warning: Some dates could not be parsed after concatenation.")
    print(missing_dates)

# Display the combined DataFrame
print("Final combined DataFrame with parsed dates:")
buli_df

Final combined DataFrame with parsed dates:


/var/folders/72/b7zxktp96cz1n4tjk3mlbj5w0000gn/T/ipykernel_11984/4226996146.py:34: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  buli_df['Date'] = pd.to_datetime(buli_df['Date'], dayfirst=True, errors='coerce')


,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,BFECAHA,IWCH,IWCD,IWCA,VCCH,VCCD,VCCA,Unnamed: 70,Unnamed: 71,Unnamed: 72
0,D1,2015-08-14,Bayern Munich,Hamburg,5,0,H,1,0,H,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,D1,2015-08-15,Augsburg,Hertha,0,1,A,0,0,D,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,D1,2015-08-15,Darmstadt,Hannover,2,2,D,1,0,H,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,D1,2015-08-15,Dortmund,M'gladbach,4,0,H,3,0,H,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,D1,2015-08-15,Leverkusen,Hoffenheim,2,1,H,1,1,D,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5575,D1,2013-05-18,Hamburg,Leverkusen,0,1,A,0,0,D,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5576,D1,2013-05-18,Hannover,Fortuna Dusseldorf,3,0,H,1,0,H,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5577,D1,2013-05-18,M'gladbach,Bayern Munich,3,4,A,3,2,H,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5578,D1,2013-05-18,Nurnberg,Werder Bremen,3,2,H,0,1,A,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [2]:
#formatting the date column to datetime format and sorting by date
#buli_df['Date'] = pd.to_datetime(buli_df['Date'])

buli_df = buli_df.sort_values(['Date']).reset_index(drop=True)
buli_df

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,BFECAHA,IWCH,IWCD,IWCA,VCCH,VCCD,VCCA,Unnamed: 70,Unnamed: 71,Unnamed: 72
0,D1,2006-08-11,Bayern Munich,Dortmund,2,0,H,1,0,H,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,D1,2006-08-12,Leverkusen,Aachen,3,0,H,2,0,H,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,D1,2006-08-12,Mainz,Bochum,2,1,H,1,0,H,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,D1,2006-08-12,M'gladbach,Cottbus,2,0,H,0,0,D,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,D1,2006-08-12,Schalke 04,Ein Frankfurt,1,1,D,1,0,H,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5575,NaN,2024-10-26,St Pauli,Wolfsburg,0,0,D,0,0,D,...,2.08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5576,NaN,2024-10-26,RB Leipzig,Freiburg,3,1,H,0,1,A,...,1.90,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5577,NaN,2024-10-27,Heidenheim,Hoffenheim,0,0,D,0,0,D,...,1.93,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5578,NaN,2024-10-27,Union Berlin,Ein Frankfurt,1,1,D,0,1,A,...,2.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
#checking for null values
buli_df.isna().sum()

Div              72
Date              0
HomeTeam          0
AwayTeam          0
FTHG              0
               ... 
VCCD           4050
VCCA           4050
Unnamed: 70    5580
Unnamed: 71    5580
Unnamed: 72    5580
Length: 168, dtype: int64

In [4]:
#dropping rows & columns with all null values
buli_df.dropna(axis=1, how='all', inplace=True) #dropped 3 columns
buli_df.dropna(axis=0, how='all',inplace=True) #0 rows dropped

In [5]:
buli_df.isna().sum()

Div           72
Date           0
HomeTeam       0
AwayTeam       0
FTHG           0
            ... 
IWCD        4223
IWCA        4223
VCCH        4050
VCCD        4050
VCCA        4050
Length: 165, dtype: int64

In [13]:
#code for including all games (first games of the season take the last games of the last season as past games)
#4 past games

import pandas as pd

buli_df_red = buli_df[['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HTHG', 'HTAG', 'HTR', 'HS', 'AS', 'HST', 'AST', 'HF', 'AF', 'HC', 'AC', 'HY', 'AY', 'HR', 'AR']]

df = buli_df_red

# Define stats dictionary with the specified columns
stats = {
    'goals': {'scored': ('FTHG', 'FTAG'), 'conceded': ('FTAG', 'FTHG')},
    'shots': {'taken': ('HS', 'AS'), 'conceded': ('AS', 'HS')},
    'shots_on_target': {'taken': ('HST', 'AST'), 'conceded': ('AST', 'HST')},
    'fouls': {'fouls': ('HF', 'AF'), 'fouled': ('AF', 'HF')},
    'corners': {'taken': ('HC', 'AC'), 'conceded': ('AC', 'HC')},
    'yellow_cards': {'received': ('HY', 'AY'), 'provoked': ('AY', 'HY')},
    'red_cards': {'received': ('HR', 'AR'), 'provoked': ('AR', 'HR')},
}

# Define `npm` for the number of past matches to consider
npm = 4

# Initialize an empty list to accumulate each row's data as a dictionary
rows_list = []

# Iterate through each row to calculate rolling stats based on home and away perspectives
for index, row in df.iterrows():
    team_h = row['HomeTeam']
    team_a = row['AwayTeam']
    date = row['Date']
    
    # Get the past `npm` games for the home team, filtered by games before the current match date
    past_matches_home = df[((df['HomeTeam'] == team_h) | (df['AwayTeam'] == team_h)) & (df['Date'] < date)]
    past_matches_home = past_matches_home.tail(npm)

    # Get the past `npm` games for the away team, filtered by games before the current match date
    past_matches_away = df[((df['HomeTeam'] == team_a) | (df['AwayTeam'] == team_a)) & (df['Date'] < date)]
    past_matches_away = past_matches_away.tail(npm)

    # Initialize a dictionary to store the calculated stats for each row
    row_stats = {
        'Date': date,
        'HomeTeam': team_h,
        'AwayTeam': team_a,
        'FTR': row['FTR'],
        'FTHG': row['FTHG'],
        'FTAG': row['FTAG'],
    }
    
    # Calculate stats for the home team based on whether they played home or away in past matches
    for stat, subcategories in stats.items():
        for subcategory, columns in subcategories.items():
            home_column, away_column = columns
            # Sum the stat when the home team was actually playing at home
            stat_home_as_home = past_matches_home.loc[past_matches_home['HomeTeam'] == team_h, home_column].sum()
            # Sum the stat when the home team was actually playing as the away team
            stat_home_as_away = past_matches_home.loc[past_matches_home['AwayTeam'] == team_h, away_column].sum()
            row_stats[f'p_home_{stat}_{subcategory}_last_{npm}'] = stat_home_as_home + stat_home_as_away
            
    # Calculate stats for the away team based on whether they played home or away in past matches
    for stat, subcategories in stats.items():
        for subcategory, columns in subcategories.items():
            home_column, away_column = columns
            # Sum the stat when the away team was actually playing at home
            stat_away_as_home = past_matches_away.loc[past_matches_away['HomeTeam'] == team_a, home_column].sum()
            # Sum the stat when the away team was actually playing as the away team
            stat_away_as_away = past_matches_away.loc[past_matches_away['AwayTeam'] == team_a, away_column].sum()
            row_stats[f'p_away_{stat}_{subcategory}_last_{npm}'] = stat_away_as_home + stat_away_as_away

    # Calculate points for the home team in the past `npm` games
    points_home = (
        (past_matches_home.loc[past_matches_home['HomeTeam'] == team_h, 'FTR'] == 'H').sum() * 3 +
        (past_matches_home.loc[past_matches_home['AwayTeam'] == team_h, 'FTR'] == 'A').sum() * 3 +
        (past_matches_home['FTR'] == 'D').sum() * 1
    )
    row_stats[f'p_home_points_last_{npm}'] = points_home
    
    # Calculate points for the away team in the past `npm` games
    points_away = (
        (past_matches_away.loc[past_matches_away['HomeTeam'] == team_a, 'FTR'] == 'H').sum() * 3 +
        (past_matches_away.loc[past_matches_away['AwayTeam'] == team_a, 'FTR'] == 'A').sum() * 3 +
        (past_matches_away['FTR'] == 'D').sum() * 1
    )
    row_stats[f'p_away_points_last_{npm}'] = points_away
    
    # Append the dictionary for this row to the list
    rows_list.append(row_stats)

# Convert the list of dictionaries to a DataFrame
rolling_stats = pd.DataFrame(rows_list)

# Display the final DataFrame with only the desired columns
print("Final DataFrame with selected initial columns and past 7 games stats:")
rolling_stats_with_first_games_of_season = rolling_stats

Final DataFrame with selected initial columns and past 7 games stats:


In [193]:
#check with csv file if code ran correct
rolling_stats_with_first_games_of_season.loc[5461]

Date                                      2024-04-14 00:00:00
HomeTeam                                            Darmstadt
AwayTeam                                             Freiburg
FTR                                                         A
p_home_goals_scored_last_7                                  6
p_home_goals_conceded_last_7                               22
p_home_shots_taken_last_7                                  85
p_home_shots_conceded_last_7                              139
p_home_shots_on_target_taken_last_7                        29
p_home_shots_on_target_conceded_last_7                     44
p_home_fouls_fouls_last_7                                  79
p_home_fouls_fouled_last_7                                 68
p_home_corners_taken_last_7                                26
p_home_corners_conceded_last_7                             41
p_home_yellow_cards_received_last_7                        19
p_home_yellow_cards_provoked_last_7                        12
p_home_r

In [77]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

rolling_stats["FTR_num"] = rolling_stats["FTR"].apply(lambda x: 1 if x == "D" else (2 if x == "H" else 3))

features_cl = rolling_stats.select_dtypes("number").drop(columns=["FTHG", "FTAG", "FTR_num"])
target_cl = rolling_stats["FTR_num"]
X_train_cl, X_test_cl, y_train_cl, y_test_cl = train_test_split(features_cl, target_cl, random_state = 0)

# Scaling
scaler = MinMaxScaler()
scaler.fit(X_train_cl)
X_train_cl_scaled = scaler.transform(X_train_cl)
X_test_cl_scaled = scaler.transform(X_test_cl)

In [79]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

## logistic regression testing the categorical ftr

# Initialize and train the logistic regression model
log_reg = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=1000)
log_reg.fit(X_train_cl_scaled, y_train_cl)

# Make predictions on the test set
y_pred = log_reg.predict(X_test_cl_scaled)

# Evaluate the model
accuracy = accuracy_score(y_test_cl, y_pred)
print(f"Accuracy: {accuracy:.2f}")

# Displaying classification report and confusion matrix for deeper insights
print("\nClassification Report:")
print(classification_report(y_test_cl, y_pred))

print("\nConfusion Matrix:")
print(confusion_matrix(y_test_cl, y_pred))

Accuracy: 0.49

Classification Report:
              precision    recall  f1-score   support

           1       0.30      0.01      0.02       358
           2       0.52      0.80      0.63       633
           3       0.44      0.44      0.44       404

    accuracy                           0.49      1395
   macro avg       0.42      0.42      0.36      1395
weighted avg       0.44      0.49      0.42      1395


Confusion Matrix:
[[  3 247 108]
 [  3 509 121]
 [  4 223 177]]


In [81]:
## KNN classifier for categorical FTR

knn = KNeighborsClassifier(n_neighbors = 140)
knn.fit(X_train_cl_scaled, y_train_cl)
knn.score(X_test_cl_scaled, y_test_cl)

0.49032258064516127

In [103]:
## Decision Tree CL

tree = DecisionTreeClassifier(max_depth=2)
tree.fit(X_train_cl_scaled, y_train_cl)

pred = tree.predict(X_test_cl)
print("Accuracy:", accuracy_score(y_test_cl, pred))
print(classification_report(y_test_cl, pred))

Accuracy: 0.4551971326164875
              precision    recall  f1-score   support

           1       0.00      0.00      0.00       358
           2       0.46      1.00      0.63       633
           3       0.40      0.00      0.01       404

    accuracy                           0.46      1395
   macro avg       0.29      0.33      0.21      1395
weighted avg       0.32      0.46      0.29      1395



/opt/anaconda3/lib/python3.12/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but DecisionTreeClassifier was fitted without feature names
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted

In [161]:
## random forest

from sklearn.ensemble import RandomForestClassifier

X_train_cl, X_test_cl, y_train_cl, y_test_cl = train_test_split(features_cl, target_cl, random_state = 0)

# Scaling
scaler = MinMaxScaler()
scaler.fit(X_train_cl)
X_train_cl_scaled = scaler.transform(X_train_cl)
X_test_cl_scaled = scaler.transform(X_test_cl)

# Initialize and train the Random Forest model
# n_estimators is the number of trees in the forest, and max_depth controls tree depth
rf_model = RandomForestClassifier(n_estimators=50, max_depth=4, random_state=42)
rf_model.fit(X_train_cl_scaled, y_train_cl)

# Make predictions on the test set
y_pred = rf_model.predict(X_test_cl_scaled)

# Evaluate the model
accuracy = accuracy_score(y_test_cl, y_pred)
print(f"Accuracy: {accuracy:.2f}")

# Displaying classification report and confusion matrix for deeper insights
print("\nClassification Report:")
print(classification_report(y_test_cl, y_pred))

print("\nConfusion Matrix:")
print(confusion_matrix(y_test_cl, y_pred))

# Optional: Feature importance analysis
feature_importances = pd.DataFrame({
    'Feature': features_cl.columns,  # Use the columns from X_cl to match the model input
    'Importance': rf_model.feature_importances_
}).sort_values(by='Importance', ascending=False)

print("\nFeature Importances:")
print(feature_importances)

Accuracy: 0.49

Classification Report:
              precision    recall  f1-score   support

           1       0.00      0.00      0.00       358
           2       0.49      0.89      0.63       633
           3       0.47      0.30      0.37       404

    accuracy                           0.49      1395
   macro avg       0.32      0.39      0.33      1395
weighted avg       0.36      0.49      0.39      1395


Confusion Matrix:
[[  0 297  61]
 [  0 561  72]
 [  0 284 120]]

Feature Importances:
                                   Feature  Importance
14              p_away_goals_scored_last_4    0.097611
2                p_home_shots_taken_last_4    0.091114
17            p_away_shots_conceded_last_4    0.075689
28                    p_home_points_last_4    0.075096
4      p_home_shots_on_target_taken_last_4    0.069509
16               p_away_shots_taken_last_4    0.069465
18     p_away_shots_on_target_taken_last_4    0.066421
3             p_home_shots_conceded_last_4    0.05708

/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [129]:
## random forest using threwhold filtering

# Filter out features below the threshold
important_features = feature_importances[feature_importances['Importance'] > importance_threshold]['Feature']
features_important = features_cl[important_features]

X_train_cl, X_test_cl, y_train_cl, y_test_cl = train_test_split(features_important, target_cl, random_state = 42)

from sklearn.ensemble import RandomForestClassifier
# Initialize and train the Random Forest model
# n_estimators is the number of trees in the forest, and max_depth controls tree depth
rf_model = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=0)
rf_model.fit(X_train_cl_scaled, y_train_cl)

# Make predictions on the test set
y_pred = rf_model.predict(X_test_cl_scaled)

# Evaluate the model
accuracy = accuracy_score(y_test_cl, y_pred)
print(f"Accuracy: {accuracy:.2f}")

# Displaying classification report and confusion matrix for deeper insights
print("\nClassification Report:")
print(classification_report(y_test_cl, y_pred))

print("\nConfusion Matrix:")
print(confusion_matrix(y_test_cl, y_pred))

# Optional: Feature importance analysis
feature_importances = pd.DataFrame({
    'Feature': features_cl.columns,  # Use the columns from X_cl to match the model input
    'Importance': rf_model.feature_importances_
}).sort_values(by='Importance', ascending=False)

print("\nFeature Importances:")
print(feature_importances)

# Define a threshold for feature importance
importance_threshold = 0.01  # Adjust based on importance scores

# Filter out features below the threshold
important_features = feature_importances[feature_importances['Importance'] > importance_threshold]['Feature']
features_important = features_cl[important_features]

Accuracy: 0.43

Classification Report:
              precision    recall  f1-score   support

           1       0.00      0.00      0.00       371
           2       0.44      0.97      0.60       609
           3       0.21      0.02      0.03       415

    accuracy                           0.43      1395
   macro avg       0.21      0.33      0.21      1395
weighted avg       0.25      0.43      0.27      1395


Confusion Matrix:
[[  0 361  10]
 [  0 592  17]
 [  1 407   7]]

Feature Importances:
                                   Feature  Importance
17            p_away_shots_conceded_last_4    0.046898
20               p_away_fouls_fouls_last_4    0.046531
21              p_away_fouls_fouled_last_4    0.046236
16               p_away_shots_taken_last_4    0.044367
3             p_home_shots_conceded_last_4    0.044193
5   p_home_shots_on_target_conceded_last_4    0.043887
7               p_home_fouls_fouled_last_4    0.043795
2                p_home_shots_taken_last_4    0.04268

In [195]:
#code for starting every season only with the 8th games, so that every game of the season that is taken into account has
# 7 past games that were played within the very same season

import pandas as pd
import numpy as np

# Sample data setup (make sure 'Date' column is in datetime format)
df['Date'] = pd.to_datetime(df['Date'])

# Step 1: Identify season breaks by detecting gaps of 2 months or more
df = df.sort_values(by='Date').reset_index(drop=True)
df['Date_Diff'] = df['Date'].diff().dt.days
season_breaks = df[df['Date_Diff'] >= 60].index  # Gaps of 60+ days signify a new season

# Step 2: Assign a season identifier
df['Season'] = 0
current_season = 1
for i in range(len(df)):
    if i in season_breaks:
        current_season += 1
    df.at[i, 'Season'] = current_season

# Drop the Date_Diff column as it's no longer needed
df = df.drop(columns=['Date_Diff'])

# Define `npm` for the number of past matches to consider
npm = 7

# Initialize an empty list to accumulate each row's data as a dictionary
rows_list = []

# Iterate through each row to calculate rolling stats based on home and away perspectives
for index, row in df.iterrows():
    team_h = row['HomeTeam']
    team_a = row['AwayTeam']
    date = row['Date']
    
    # Get the past `npm` games for the home team within the same season
    past_matches_home = df[((df['HomeTeam'] == team_h) | (df['AwayTeam'] == team_h)) & 
                           (df['Date'] < date) & 
                           (df['Season'] == row['Season'])]
    past_matches_home = past_matches_home.tail(npm)

    # Get the past `npm` games for the away team within the same season
    past_matches_away = df[((df['HomeTeam'] == team_a) | (df['AwayTeam'] == team_a)) & 
                           (df['Date'] < date) & 
                           (df['Season'] == row['Season'])]
    past_matches_away = past_matches_away.tail(npm)

    # Initialize a dictionary to store the calculated stats for each row
    row_stats = {
        'Date': date,
        'HomeTeam': team_h,
        'AwayTeam': team_a,
        'FTR': row['FTR'],
        'FTHG': row['FTHG'],  # Include Full Time Home Goals directly
        'FTAG': row['FTAG'],  # Include Full Time Away Goals directly
        'Season': row['Season']
    }
    
    # Calculate stats for the home team based on whether they played home or away in past matches
    for stat, subcategories in stats.items():
        for subcategory, columns in subcategories.items():
            home_column, away_column = columns
            # Sum the stat when the home team was actually playing at home
            stat_home_as_home = past_matches_home.loc[past_matches_home['HomeTeam'] == team_h, home_column].sum()
            # Sum the stat when the home team was actually playing as the away team
            stat_home_as_away = past_matches_home.loc[past_matches_home['AwayTeam'] == team_h, away_column].sum()
            row_stats[f'p_home_{stat}_{subcategory}_last_{npm}'] = stat_home_as_home + stat_home_as_away
            
    # Calculate stats for the away team based on whether they played home or away in past matches
    for stat, subcategories in stats.items():
        for subcategory, columns in subcategories.items():
            home_column, away_column = columns
            # Sum the stat when the away team was actually playing at home
            stat_away_as_home = past_matches_away.loc[past_matches_away['HomeTeam'] == team_a, home_column].sum()
            # Sum the stat when the away team was actually playing as the away team
            stat_away_as_away = past_matches_away.loc[past_matches_away['AwayTeam'] == team_a, away_column].sum()
            row_stats[f'p_away_{stat}_{subcategory}_last_{npm}'] = stat_away_as_home + stat_away_as_away

    # Calculate points for the home team in the past `npm` games
    points_home = (
        (past_matches_home.loc[past_matches_home['HomeTeam'] == team_h, 'FTR'] == 'H').sum() * 3 +
        (past_matches_home.loc[past_matches_home['AwayTeam'] == team_h, 'FTR'] == 'A').sum() * 3 +
        (past_matches_home['FTR'] == 'D').sum() * 1
    )
    row_stats[f'p_home_points_last_{npm}'] = points_home
    
    # Calculate points for the away team in the past `npm` games
    points_away = (
        (past_matches_away.loc[past_matches_away['HomeTeam'] == team_a, 'FTR'] == 'H').sum() * 3 +
        (past_matches_away.loc[past_matches_away['AwayTeam'] == team_a, 'FTR'] == 'A').sum() * 3 +
        (past_matches_away['FTR'] == 'D').sum() * 1
    )
    row_stats[f'p_away_points_last_{npm}'] = points_away
    
    # Append the dictionary for this row to the list
    rows_list.append(row_stats)

# Convert the list of dictionaries to a DataFrame
buli_df = pd.DataFrame(rows_list)

# Filter out the first `npm` games of each season from the final output
buli_df_first_games_skipped = buli_df.groupby('Season').apply(lambda x: x.iloc[npm:]).reset_index(drop=True)

# Display the final DataFrame with selected columns and past 7 games stats
buli_df_first_games_skipped

/var/folders/72/b7zxktp96cz1n4tjk3mlbj5w0000gn/T/ipykernel_1111/2702053440.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Date'] = pd.to_datetime(df['Date'])
/var/folders/72/b7zxktp96cz1n4tjk3mlbj5w0000gn/T/ipykernel_1111/2702053440.py:101: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  buli_df_first_games_skipped = buli_df.groupby('Season').apply(lambda x: x.iloc[npm:]).reset_index(drop=True)


,Date,HomeTeam,AwayTeam,FTR,FTHG,FTAG,Season,p_home_goals_scored_last_7,p_home_goals_conceded_last_7,p_home_shots_taken_last_7,...,p_away_fouls_fouls_last_7,p_away_fouls_fouled_last_7,p_away_corners_taken_last_7,p_away_corners_conceded_last_7,p_away_yellow_cards_received_last_7,p_away_yellow_cards_provoked_last_7,p_away_red_cards_received_last_7,p_away_red_cards_provoked_last_7,p_home_points_last_7,p_away_points_last_7
0,2006-08-13,Wolfsburg,Hertha,D,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2006-08-13,Hannover,Werder Bremen,A,2,4,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2006-08-18,Nurnberg,M'gladbach,H,1,0,1,3,0,11,...,11,24,3,5,0,4,0,1,3,3
3,2006-08-19,Cottbus,Hamburg,D,2,2,1,0,2,11,...,29,22,11,2,3,2,0,0,0,1
4,2006-08-19,Aachen,Schalke 04,A,0,1,1,0,3,8,...,17,19,7,5,1,2,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5428,2024-10-26,Werder Bremen,Leverkusen,D,2,2,21,12,14,72,...,59,67,61,22,16,13,0,1,11,14
5429,2024-10-26,RB Leipzig,Freiburg,H,3,1,21,11,2,85,...,58,77,38,27,9,11,0,0,17,15
5430,2024-10-27,Union Berlin,Ein Frankfurt,D,1,1,21,8,4,84,...,69,62,32,44,9,9,0,0,14,13
5431,2024-10-27,Heidenheim,Hoffenheim,D,0,0,21,12,11,89,...,74,92,32,42,18,13,1,1,9,7


In [ ]:
#seasons dont fit yet, 2 month definition not really working at some points apparently